<a href="https://colab.research.google.com/github/puneethkumarvydya/Capstone-Project/blob/Ganesh's-Branch/Capstone_Project_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Pneumonia detection from chest X-Ray images 

# Project Description


In this capstone project, the goal is to build a pneumonia detection system, to locate the position of inflammation in an image. Tissues with sparse material, such as lungs which are full of air, do not absorb the X-rays and appear black in the image. Dense tissues such as bones absorb X-rays and appear white in the image. While we are theoretically detecting “lung opacities”, there are lung opacities that are not pneumonia related. In the data, some of these are labeled “Not Normal No Lung Opacity”. This extra third class indicates that while pneumonia was determined not to be present, there was nonetheless some type of abnormality on the image and oftentimes this finding may mimic the appearance of true pneumonia. Dicom original images: - Medical images are stored in a special format called DICOM files (*.dcm). They contain a combination of header metadata as well as underlying raw image arrays for pixel data. Details about the data and dataset files are given in below link, https://www.kaggle.com/c/rsna-pneumonia-detection-challenge/data

# Pre-Processing, Data Visualization, EDA
 Exploring the given Data files, classes and images of different classes.

 Dealing with missing values

 Visualization of different classes

 Analysis from the visualization of different classes.

# Model Building
 Building a pneumonia detection model starting from basic CNN and then improving upon it.
 Train the model

 To deal with large training time, save the weights so that you can use them when training the

model for the second time without starting from scratch.

 **Test the model and report as per evaluation metrics**

 Try different models

 Set different hyper parameters, by trying different optimizers, loss functions, epochs, learning

rate, batch size, checkpointing, early stopping etc. for these models to fine-tune them

 Report evaluation metrics for these models along with your observation on how changing

different hyper parameters leads to change in the final evaluation metric.

In [3]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Let’s start with loading all the libraries and dependencies.

In [5]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

seed = 232
np.random.seed(seed)
#tf.set_random_seed(seed)

In [6]:
os.chdir('/content/drive/My Drive/projects/capstone_project/Capestone/')
#path = ('.')
os.listdir()

['Pneumonia Detection Challenge Brief.pdf',
 'Interim Report Format.pdf',
 'Final Report Format.pdf',
 'rsna-pneumonia-detection-challenge.zip',
 'GCP Credits Request Link - RSNA.txt',
 'stage_2_detailed_class_info.csv',
 'stage_2_sample_submission.csv',
 'stage_2_test_images',
 'stage_2_train_images',
 'stage_2_train_labels.csv']

# All our images are in the zip file so we need to extract the contents of this file

In [ ]:
#images_zip_path = project_path + 'Aligned Face Dataset from Pinterest.zip' #"images.zip"
from zipfile import ZipFile
with ZipFile("rsna-pneumonia-detection-challenge.zip",'r') as z:
  z.extractall()

## DICOM images are generally used to store medical images. They are highly informative. Along with image data it stores lots of key patient information, such as, patient’s name, age, sex, doctor’s name etc.

## ##If you just want to preview dicom images without extracting any information then use the following code. You need to install pydicom python package, you can install using **pip install pydicom** -

In [ ]:
pip install pydicom

In [ ]:
import pydicom as dicom
import matplotlib.pylab as plt

# specify your image path
#image_path = 'data/train_images/sample.dcm'
#ds = dicom.dcmread(image_path)

#plt.imshow(ds.pixel_array)

#If you want to convert the image to png/jpg then you can do the following -

In [ ]:
import pydicom as dicom
import cv2   

# specify your image path
image_path = 'data/train_images/sample.dcm'
ds = dicom.dcmread(image_path)

pixel_array_numpy = ds.pixel_array

image_format = '.jpg' # or '.png'
image_path = image_path.replace('.dcm', image_format)

cv2.imwrite(image_path, pixel_array_numpy)